# 장애인콜택시 대기시간 예측
## 단계1.데이터 전처리

## 0.미션

* 1.데이터를 탐색하며 정보 획득
    * 데이터는 2015-01-01 ~ 2022-12-31 까지의 서울 장애인 콜택시 운행 정보입니다. 우리는 평균대기시간을 예측하고자 합니다. 
    * 요일, 월, 계절, 연도별 운행 정보에 어떤 주기와 변화가 있는지 탐색해 봅시다.
* 2.분석을 위한 데이터 구조 만들기
    * 문제정의 : 
        * 전 날 콜택시 운행이 종료되었을 때, 다음 날 대기시간을 예측하고자 합니다.

        * 만약 다음 날 대기시간을 예측할 수 있다면, 일정 범위 내에서 배차를 조절할 수 있을 뿐만 아니라, 향후 교통약자의 이동 편의 증진을 위한 정책 수립 및 개선에 기여할 수 있습니다. 
    * 이를 위한 데이터 구조를 만들어 봅시다.
        * 분석 단위는 일별 데이터 입니다.
        * 주어진 데이터 : 장애인 콜택시 운행 정보, 서울시 날씨
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전 날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
        * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙여서 만듭시다.
        * 휴무일 데이터는 패키지를 통해서 다운받아 사용합니다.
    * Feature Engineering
        * 대기시간에 영향을 주는 요인을 도출하고(가설수립) 이를 feature로 생성합시다.
        * 주어진 그대로의 데이터가 아닌 새로운 feature를 생성해 봅시다.
            * 날짜와 관련된 feature : 요일, 월, 계절 ... 
            * 시계열 특성이 반영된 feature : 최근 7일간의 평균 대기시간 ...




## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 다음의 두가지 방법 중 하나를 선택하여 폴더를 준비하고 데이터를 로딩하시오.
        * 1) 로컬 수행(Ananconda)
            * 제공된 압축파일을 다운받아 압축을 풀고
            * anaconda의 root directory(보통 C:/Users/< ID > 에 project 폴더를 만들고, 복사해 넣습니다.
        * 2) 구글콜랩
            * 구글 드라이브 바로 밑에 project 폴더를 만들고, 
            * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 설치 및 로딩
        * requirements.txt 파일로 부터 라이브러리 설치
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다. 
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

#### 1) 로컬 수행(Anaconda)
* project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

### (2) 라이브러리 설치 및 불러오기

#### 1) 라이브러리 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

# 더 필요한 라이브러리를 아래에 추가합니다.

### (3) 데이터 불러오기
* 주어진 데이터셋
    * 장애인 콜택시 운행 정보 : open_data.csv
    * 날씨 데이터 : weather.csv

#### 1) 데이터로딩

In [2]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi = pd.read_csv('open_data.csv')
weather = pd.read_csv('weather.csv')

#### 2) 기본 정보 조회

In [3]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     2922 non-null   object 
 1   차량운행    2922 non-null   int64  
 2   접수건     2922 non-null   int64  
 3   탑승건     2922 non-null   int64  
 4   평균대기시간  2922 non-null   float64
 5   평균요금    2922 non-null   int64  
 6   평균승차거리  2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             4018 non-null   object 
 1   temp_max         4018 non-null   float64
 2   temp_min         4018 non-null   float64
 3   rain(mm)         4018 non-null   float64
 4   humidity_max(%)  4018 non-null   float64
 5   humidity_min(%)  4018 non-null   float64
 6   sunshine(MJ/m2)  4018 non-null   float64
dtypes: float64(6), object(1)
memory usage: 219.9+ KB


#### 3) 칼럼이름을 영어로 변경
* 꼭 필요한 작업은 아니지만, 데이터를 편리하게 다루고 차트에서 불필요한 경고메시지를 띄우지 않게 하기 위해 영문으로 변경하기를 권장합니다.


In [5]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi_re = taxi.copy()
taxi_re.rename(columns = {'기준일' : 'date',	'차량운행' : 'drive',	
                       '접수건' : 'call_num', '탑승건' : 'ride_num', 
                       '평균대기시간' : 'waiting_time','평균요금' : 'fee','평균승차거리' : 'dis'}, inplace = True)

taxi_re.head()

,date,drive,call_num,ride_num,waiting_time,fee,dis
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


## 2.데이터 기본 탐색

* **세부요구사항**
    * 날짜 요소에 따라 각 정보의 패턴을 조회 합니다.
        * 일별, 요일별, 주차별, 월별, 연도별
        * 접수건, 탑승건, 거리, 요금, 대기시간 등
    * 제시된 범위 외에 가능하다면 추가 탐색을 시도합니다.

### (1) 주기별 분석을 위해서 날짜 변수 추가하기
* data를 복사합니다.
* 복사한 df에 요일, 주차, 월, 연도 등을 추가합니다.

In [6]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi_re['date'] = pd.to_datetime(taxi_re['date'])
taxi_re['weekday'] = taxi_re['date'].dt.weekday

# 요일 추가
taxi_re['weekday'] = taxi_re['weekday'].map({0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'})

taxi_re['day'] = taxi_re['date'].dt.day
taxi_re['month'] = taxi_re['date'].dt.month
taxi_re['year'] = taxi_re['date'].dt.year
taxi_re.tail()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year
2917,2022-12-27,669,5635,4654,44.4,2198,8178,화,27,12,2022
2918,2022-12-28,607,5654,4648,44.8,2161,7882,수,28,12,2022
2919,2022-12-29,581,5250,4247,52.5,2229,8433,목,29,12,2022
2920,2022-12-30,600,5293,4200,38.3,2183,8155,금,30,12,2022
2921,2022-12-31,263,2167,1806,33.7,2318,9435,토,31,12,2022


* 차량 운행수

In [7]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_drive = taxi_re.groupby('month')['drive'].mean()
month_drive.head()

month
1    379.286290
2    373.486726
3    398.758065
4    416.187500
5    413.556452
Name: drive, dtype: float64

* 접수건, 탑승건

In [92]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_call_num = taxi_re.groupby('month')['call_num'].mean()
month_call_num

month
1     3681.826613
2     3582.097345
3     3691.330645
4     3964.479167
5     3910.572581
6     4001.304167
7     4099.637097
8     4014.963710
9     3962.508333
10    4007.709677
11    4243.991667
12    3931.556452
Name: call_num, dtype: float64

In [93]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_ride_num = taxi_re.groupby('month')['ride_num'].mean()
month_ride_num.head()

month
1    3092.262097
2    3032.075221
3    3168.822581
4    3358.558333
5    3278.435484
Name: ride_num, dtype: float64

* 대기시간

In [94]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_waiting_time = taxi_re.groupby('month')['waiting_time'].mean()
month_waiting_time

month
1     33.393952
2     34.419469
3     35.182258
4     39.628750
5     41.181452
6     40.381667
7     40.033468
8     38.568952
9     42.364167
10    44.480242
11    47.063333
12    46.713306
Name: waiting_time, dtype: float64

* 운임

In [95]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_fee = taxi_re.groupby('month')['fee'].mean()
month_fee.head()

month
1    2279.616935
2    2296.623894
3    2291.600806
4    2310.712500
5    2326.209677
Name: fee, dtype: float64

* 이동거리

In [96]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
month_dis = taxi_re.groupby('month')['dis'].mean()
month_dis.head()

month
1    9048.161290
2    9221.039823
3    9129.588710
4    9313.458333
5    9462.447581
Name: dis, dtype: float64

* 차량 운행수

In [97]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
year_drive = taxi_re.groupby('year')['drive'].mean()
year_drive

year
2015    381.468493
2016    390.314208
2017    405.397260
2018    397.758904
2019    399.068493
2020    416.521858
2021    468.271233
2022    490.898630
Name: drive, dtype: float64

* 접수건, 탑승건

In [98]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
year_call_num = taxi_re.groupby('year')['call_num'].mean()
year_call_num

year
2015    3750.531507
2016    4005.087432
2017    4184.745205
2018    4077.473973
2019    4104.358904
2020    3137.480874
2021    3796.821918
2022    4350.304110
Name: call_num, dtype: float64

In [99]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
year_ride_num = taxi_re.groupby('year')['ride_num'].mean()
year_ride_num

year
2015    3268.673973
2016    3418.183060
2017    3471.709589
2018    3209.380822
2019    3230.279452
2020    2656.314208
2021    3344.109589
2022    3673.863014
Name: ride_num, dtype: float64

In [100]:
year_waiting_time = taxi_re.groupby('year')['waiting_time'].mean()
year_waiting_time

year
2015    34.059452
2016    38.272404
2017    44.143014
2018    56.322192
2019    53.455616
2020    28.045082
2021    30.720822
2022    37.466027
Name: waiting_time, dtype: float64

## 3.데이터 구조 만들기

* **세부요구사항**
    * 조건 : 
        * 목표 : 전날 저녁, 다음날 평균 대기시간을 예측하고자 합니다.
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
    * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙입니다.

### (1) target 만들기
* 예측하는 날짜, 대기시간(target)으로 기준을 잡습니다.

In [8]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 대기시간(waiting time)을 오늘의 데이터를 활용하여 예측 해야하는 대상(target)으로 설정
target = 'waiting_time'

### (2) 날씨 데이터 붙이기
* merge를 활용합니다. 기준은 운행정보 입니다.

In [9]:
weather.rename(columns = {'Date' : 'date'}, inplace = True)
weather.tail()

,date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
4013,2022-12-27,3.0,-7.3,0.0,86.0,51.0,10.25
4014,2022-12-28,-0.3,-5.4,0.1,92.0,40.0,10.86
4015,2022-12-29,1.7,-7.8,0.0,71.0,34.0,10.88
4016,2022-12-30,2.1,-4.0,0.0,87.0,38.0,10.84
4017,2022-12-31,-4.4,-4.4,0.0,66.0,66.0,0.00


In [10]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 실제 날씨 데이터를 전일에 발표된 예보데이터로 판단
weather['date'] = pd.to_datetime(weather['date'])
data = pd.merge(taxi_re, weather, on = 'date', how = 'inner')
data.head()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2015-01-01,213,1023,924,23.2,2427,10764,목,1,1,2015,-4.3,-9.8,0.0,52.0,33.0,9.79
1,2015-01-02,420,3158,2839,17.2,2216,8611,금,2,1,2015,-2.0,-8.9,0.0,63.0,28.0,9.07
2,2015-01-03,209,1648,1514,26.2,2377,10198,토,3,1,2015,2.4,-9.2,0.0,73.0,37.0,8.66
3,2015-01-04,196,1646,1526,24.5,2431,10955,일,4,1,2015,8.2,0.2,0.0,89.0,58.0,5.32
4,2015-01-05,421,4250,3730,26.2,2214,8663,월,5,1,2015,7.9,-0.9,0.0,95.0,52.0,6.48


In [11]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 날씨 데이터를 한 줄씩 아래로 밀기
data_modi = data.copy()
shift_col = ['temp_max','temp_min', 'rain(mm)', 'humidity_max(%)', 'humidity_min(%)', 'sunshine(MJ/m2)']
data_modi[shift_col] = data[shift_col].shift(-1)
data_modi.tail()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
2917,2022-12-27,669,5635,4654,44.4,2198,8178,화,27,12,2022,-0.3,-5.4,0.1,92.0,40.0,10.86
2918,2022-12-28,607,5654,4648,44.8,2161,7882,수,28,12,2022,1.7,-7.8,0.0,71.0,34.0,10.88
2919,2022-12-29,581,5250,4247,52.5,2229,8433,목,29,12,2022,2.1,-4.0,0.0,87.0,38.0,10.84
2920,2022-12-30,600,5293,4200,38.3,2183,8155,금,30,12,2022,-4.4,-4.4,0.0,66.0,66.0,0.00
2921,2022-12-31,263,2167,1806,33.7,2318,9435,토,31,12,2022,NaN,NaN,NaN,NaN,NaN,NaN


### (3) 새로운 feature를 생성해 봅시다.
* 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 그외 새로운 feature 도출 : 최소 2개 이상
    * 예 : 공휴일, 최근 7주일간의 평균 대기시간, 탑승률 등

#### 1) 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 요일 이름, 계절이름, 월 이름으로 만드는 경우에는, 변수를 pd.Categorical로 범주형을 만들면서 순서를 지정하는 것이 이후 그래프를 그릴 때 순서대로 표현할 수 있습니다.


In [12]:
# data0['weekday'] = data0['Date'].dt.day_name()
# data0['weekday'] = pd.Categorical(data0['weekday'], 
#                                   categories=['Monday','Tuesday','Wednesday'ㄴㄴㄴㄴㄴ,'Thursday','Friday','Saturday','Sunday'])
# data0['month'] = data0['Date'].dt.month

# data0['season'] = np.where(data0['month'].isin([3,4,5]), 'Spring',
#                            np.where(data0['month'].isin([6,7,8]), 'Summer',
#                                     np.where(data0['month'].isin([9,10,11]), 'Fall', 'Winter')))
# data0['season'] = pd.Categorical(data0['season'], categories=['Spring','Summer','Fall','Winter'])
                                     
# data0['year'] = data0['Date'].dt.year
# data0

# 계절 추가하기
data_modi['season'] = np.where(data_modi['month'].isin([3,4,5]), 'Spring',
                               np.where(data_modi['month'].isin([6,7,8]), 'Summer',
                                    np.where(data_modi['month'].isin([9,10,11]), 'Fall', 'Winter')))
data_modi['season'] = pd.Categorical(data_modi['season'], categories=['Spring','Summer','Fall','Winter'])
data_modi.head()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),season
0,2015-01-01,213,1023,924,23.2,2427,10764,목,1,1,2015,-2.0,-8.9,0.0,63.0,28.0,9.07,Winter
1,2015-01-02,420,3158,2839,17.2,2216,8611,금,2,1,2015,2.4,-9.2,0.0,73.0,37.0,8.66,Winter
2,2015-01-03,209,1648,1514,26.2,2377,10198,토,3,1,2015,8.2,0.2,0.0,89.0,58.0,5.32,Winter
3,2015-01-04,196,1646,1526,24.5,2431,10955,일,4,1,2015,7.9,-0.9,0.0,95.0,52.0,6.48,Winter
4,2015-01-05,421,4250,3730,26.2,2214,8663,월,5,1,2015,4.1,-7.4,3.4,98.0,29.0,10.47,Winter


#### 2) 공휴일 정보
* workalendar 패키지를 설치하고, 대한민국 공휴일 정보를 끌어와 봅시다.

* 간단 사용법

In [13]:
from workalendar.asia import SouthKorea
cal = SouthKorea()
pd.DataFrame(cal.holidays(2023))

,0,1
0,2023-01-01,New year
1,2023-01-21,Korean New Year's Day
2,2023-01-22,Korean New Year's Day
3,2023-01-23,Korean New Year's Day
4,2023-03-01,Independence Day
5,2023-05-05,Children's Day
6,2023-05-26,Buddha's Birthday
7,2023-06-06,Memorial Day
8,2023-08-15,Liberation Day
9,2023-09-28,Midautumn Festival


* 휴무일 데이터셋 만들기 2015 ~ 2022
* 실제로 휴무일에 해당하지만 workalendar 라이브러리에 없는 날짜는 직접 추가해봅시다.
    * 휴무일 장애인 콜택시의 접수건 변화에 대한 특징을 찾아 이를 바탕으로 데이터를 조회하여 찾아볼 수 있음

In [14]:
from workalendar.asia import SouthKorea

cal = SouthKorea()
holiday = pd.DataFrame()
for y in range(2015, 2023) :
    holiday = pd.concat([holiday, pd.DataFrame(cal.holidays(y))], axis = 0)

holiday.columns = ['date', 'holiday']
holiday['date'] = pd.to_datetime(holiday['date'])
holiday['holiday'] = 1
holiday

,date,holiday
0,2015-01-01,1
1,2015-02-18,1
2,2015-02-19,1
3,2015-02-20,1
4,2015-03-01,1
...,...,...
10,2022-09-10,1
11,2022-09-11,1
12,2022-10-03,1
13,2022-10-09,1


* 기존 데이터에 휴무일 정보 결합하기.
* 휴무일이 아닌 날짜는 0으로 저장하시오.

In [15]:
data_modi = pd.merge(data_modi, holiday, how = 'left')
data_modi = data_modi.fillna({'holiday':0})

In [16]:
data_modi.head()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),season,holiday
0,2015-01-01,213,1023,924,23.2,2427,10764,목,1,1,2015,-2.0,-8.9,0.0,63.0,28.0,9.07,Winter,1.0
1,2015-01-02,420,3158,2839,17.2,2216,8611,금,2,1,2015,2.4,-9.2,0.0,73.0,37.0,8.66,Winter,0.0
2,2015-01-03,209,1648,1514,26.2,2377,10198,토,3,1,2015,8.2,0.2,0.0,89.0,58.0,5.32,Winter,0.0
3,2015-01-04,196,1646,1526,24.5,2431,10955,일,4,1,2015,7.9,-0.9,0.0,95.0,52.0,6.48,Winter,0.0
4,2015-01-05,421,4250,3730,26.2,2214,8663,월,5,1,2015,4.1,-7.4,3.4,98.0,29.0,10.47,Winter,0.0


#### 3) 7일 이동평균 대기시간
* rolling().mean() 사용

In [17]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data_modi['7_day_avg_waiting_time'] = data_modi['waiting_time'].rolling(window=7).mean()
data_modi.head(20)

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,year,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),season,holiday,7_day_avg_waiting_time
0,2015-01-01,213,1023,924,23.2,2427,10764,목,1,1,2015,-2.0,-8.9,0.0,63.0,28.0,9.07,Winter,1.0,NaN
1,2015-01-02,420,3158,2839,17.2,2216,8611,금,2,1,2015,2.4,-9.2,0.0,73.0,37.0,8.66,Winter,0.0,NaN
2,2015-01-03,209,1648,1514,26.2,2377,10198,토,3,1,2015,8.2,0.2,0.0,89.0,58.0,5.32,Winter,0.0,NaN
3,2015-01-04,196,1646,1526,24.5,2431,10955,일,4,1,2015,7.9,-0.9,0.0,95.0,52.0,6.48,Winter,0.0,NaN
4,2015-01-05,421,4250,3730,26.2,2214,8663,월,5,1,2015,4.1,-7.4,3.4,98.0,29.0,10.47,Winter,0.0,NaN
5,2015-01-06,417,3991,3633,23.6,2211,8545,화,6,1,2015,-1.0,-8.8,0.0,42.0,24.0,10.12,Winter,0.0,NaN
6,2015-01-07,410,4085,3676,24.7,2230,8646,수,7,1,2015,-0.2,-9.2,0.0,62.0,27.0,10.09,Winter,0.0,23.657143
7,2015-01-08,419,4030,3728,21.2,2231,8683,목,8,1,2015,3.2,-6.8,0.0,78.0,38.0,8.74,Winter,0.0,23.371429
8,2015-01-09,424,4167,3813,21.8,2215,8506,금,9,1,2015,4.3,-5.5,0.0,81.0,33.0,9.41,Winter,0.0,24.028571
9,2015-01-10,215,1916,1645,41.2,2447,11123,토,10,1,2015,2.3,-4.0,0.0,83.0,39.0,9.60,Winter,0.0,26.171429


#### 4) 탑승률

In [18]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
data_modi['ride_num/call_num'] = data_modi['ride_num'] / data_modi['call_num'] * 100
data_modi.head()

,date,drive,call_num,ride_num,waiting_time,fee,dis,weekday,day,month,...,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),season,holiday,7_day_avg_waiting_time,ride_num/call_num
0,2015-01-01,213,1023,924,23.2,2427,10764,목,1,1,...,-2.0,-8.9,0.0,63.0,28.0,9.07,Winter,1.0,NaN,90.322581
1,2015-01-02,420,3158,2839,17.2,2216,8611,금,2,1,...,2.4,-9.2,0.0,73.0,37.0,8.66,Winter,0.0,NaN,89.898670
2,2015-01-03,209,1648,1514,26.2,2377,10198,토,3,1,...,8.2,0.2,0.0,89.0,58.0,5.32,Winter,0.0,NaN,91.868932
3,2015-01-04,196,1646,1526,24.5,2431,10955,일,4,1,...,7.9,-0.9,0.0,95.0,52.0,6.48,Winter,0.0,NaN,92.709599
4,2015-01-05,421,4250,3730,26.2,2214,8663,월,5,1,...,4.1,-7.4,3.4,98.0,29.0,10.47,Winter,0.0,NaN,87.764706


## 4.데이터 저장
* **세부요구사항**
    * joblib 을 사용하여 작업 경로에 정리한 데이터프레임을 저장합니다.
        * 저장파일이름 : data1.pkl

In [19]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
import joblib
joblib.dump(data_modi, 'data1.pkl')

['data1.pkl']